In [72]:
from datetime import datetime, timedelta, date, time
from lxml import html
import requests
import time as t
import os
import csv
import pandas as pd
import numpy as np

pd.set_option('display.max_columns', None)


__author__ = 'aituarov'


DB_LOADER_DIR = 'C:\\DEV\\output\\'
DATAFILES_DIR = 'C:\\DEV\\output\\'


post_fix = "jepx_spot_market_trading_result"



def download_files():
    url = 'http://www.jepx.org/english/market/'
    response = requests.get(url)
    page = html.fromstring(response.text)
    table_body = page.xpath('//table[@class="tableStyle-a styleB"]/tr[position()>1]')
    
    
    today = datetime.today()
    if today.date() > datetime.strptime(today.strftime('%Y-04-01'), '%Y-%m-%d').date():
        cnt_download = 1
    else:
        cnt_download = 2
        
    for file_index in range(cnt_download):
        tr = table_body[file_index]
        file_url = url + tr.xpath('./td/a')[0].get('href')
        file_name = file_url.split('/')[-1]

        resp = requests.get(file_url)
        text = translate_text(resp.text)
        print(file_name)

        with open(DATAFILES_DIR + file_name, 'w') as output:
            output.write(text)
            output.close()

            print(' downloaded')
    print('All files downloaded')
            
            
def gen_output_csv():
    fnames = [f for f in os.listdir(DATAFILES_DIR) if f.startswith('spot_')]
    for f_id, fname in enumerate(fnames):
        data = pd.DataFrame(None)
        print("Reading " + fname + "...")
        with open(DB_LOADER_DIR + fname, newline='') as csv_file:
            csv_reader = csv.reader(csv_file, delimiter=',')
            header = next(csv_reader)    
            for line in csv_reader:
                data = data.append([line])
        data.rename(columns=lambda x: header[x], inplace=True)
        data = data.replace('', np.nan).dropna(how='all', axis=1)
        data['date'] = data['date'] + '_' + data['time code'].map(lambda x:(datetime.combine(date.today(), time(0,0))+(int(x)-1)*timedelta(minutes=30)).time().strftime('%H:%M'))
        data = data.drop(columns='time code')
        data = pd.melt(data, id_vars=['date'], value_vars = list(data.columns)[1:], var_name='name', value_name='value')

        write_stdldr(data,f_id)

    print('End.')
        
        
        
        
        
        
def write_stdldr(data,f_id):
    print("Writing to new csv file...")

    data['name'] = data['name'].map(lambda x: x.replace(' ','_').replace('(','').replace(')','').replace('/','_per_') + '_' + post_fix)

    data['effective_date'] = ""

    data['start_date'] = data['date'].map(lambda x: datetime.strptime(x, '%Y/%m/%d_%H:%M').strftime('%Y-%m-%d_%H:%M'))

    data['end_date'] = ""

    data['table'] = 'spot'

    data = data[["name","effective_date","start_date","end_date","value","table"]]

    outfile = DATAFILES_DIR + "jepx_("+str(f_id)+")_"+ datetime.today().strftime('%Y_%m_%d_%H%M%S') +".csv"

    data.to_csv(outfile, sep=';',index=False)
    
    print(outfile + " is ready")

            
            
  

        
        
        
def translate_text(str):

    dic = {'年月日': 'date', 
           '時刻コード': 'time code', 
           '売り入札量(kWh)': 'sell bid amount (kWh)', 
           '買い入札量(kWh)': 'buy bid amount (kWh)', 
           '約定総量(kWh)': 'contracted total amount (kWh)', 
           'システムプライス(円/kWh)': 'system price (yen/kWh)', 
           'エリアプライス北海道(円/kWh)': 'area price Hokkaido (yen/kWh)', 
           'エリアプライス東北(円/kWh)': 'area price Tohoku (yen/ kWh)', 
           'エリアプライス東京(円/kWh)': 'area price Tokyo (yen/kWh)', 
           'エリアプライス中部(円/kWh)': 'area price Chubu (yen/kWh)', 
           'エリアプライス北陸(円/kWh)': 'area price Hokuriku (yen/kWh)', 
           'エリアプライス関西(円/kWh)': 'area price Kansai (yen/kWh)', 
           'エリアプライス中国(円/kWh)': 'area price China (yen/kWh)', 
           'エリアプライス四国(円/kWh)': 'area price Shikoku (yen/kWh)', 
           'エリアプライス九州(円/kWh)': 'area price Kyushu (yen/kWh)', 
           'α上限値×スポット・時間前平均価格(円/kWh)': 'upper limit x spot-average price before hour (yen/kWh)', 
           'α下限値×スポット・時間前平均価格(円/kWh)': 'lower limit x spot-average price before hour (yen/kWh)', 
           'α速報値×スポット・時間前平均価格(円/kWh)': 'preliminary value x spot-hourly average price (yen/kWh)', 
           'α確報値×スポット・時間前平均価格(円/kWh)': 'confirmed value x spot-hourly average price (yen/kWh)', 
           '回避可能原価全国値(円/kWh)': 'national avoidable cost (JPY/kWh)', 
           '回避可能原価北海道(円/kWh)': 'avoidable cost Hokkaido (JPY/kWh)', 
           '回避可能原価東北(円/kWh)': 'avoidable cost Tohoku (JPY/kWh)', 
           '回避可能原価東京(円/kWh)': 'avoidable cost Tokyo (JPY/kWh)', 
           '回避可能原価中部(円/kWh)': 'avoidable cost Chubu (JPY/kWh)', 
           '回避可能原価北陸(円/kWh)': 'avoidable cost Hokuriku (JPY/kWh)', 
           '回避可能原価関西(円/kWh)': 'avoidable costs Kansai (JPY/kWh)', 
           '回避可能原価中国(円/kWh)': 'avoidable costs China (JPY/kWh)', 
           '回避可能原価四国(円/kWh)': 'avoidable costs Shikoku (JPY/kWh)', 
           '回避可能原価九州(円/kWh)': 'avoidable costs Kyushu (JPY/kWh)',
           'スポット・時間前平均価格(円/kWh)': 'Spot-average price before hour (yen/kWh)',
           '\r': ''
          }
    
    for d in dic:
        str = str.replace(d,dic[d])

    return str





def main():
    print('Starting to download files')
    download_files()
    t.sleep(3)
    print('Starting to generate output csv')
    gen_output_csv()



    
    
if __name__ == '__main__':
    main()

Starting to download files
spot_2020.csv
 downloaded
All files downloaded
Starting to generate output csv
Reading spot_2020.csv...
Writing to new csv file...
C:\DEV\output\jepx_(0)_2020_07_05_214743.csv is ready
End.


In [57]:
from datetime import time, timedelta, datetime, date
arr = []
for i in range(1, 49):
    arr.append(i)
    
print(arr)

time_arr = []

time_cr = datetime.combine(date.today(), time(0, 0))
# datetime.combine(date.today(), time(23, 55))
print(time_cr)

for i in arr:
#     time_cur = time_cr + i*timedelta(minutes=30)
    time_arr.append((datetime.combine(date.today(), time(0, 0)) + (i-1)*timedelta(minutes=30)).time().strftime('%H:%M'))
#     (datetime.combine(date.today(), time(0,0)) + (x-1)*timedelta(minutes=30))
    
print(time_arr)

# current_date_str = '2020/04/01'

# current_date_dt = datetime.strptime(current_date_str, '%Y/%m/%d')

# for i in arr:
#     time = current_date_dt + i*timedelta(minutes=30)
#     print(time)
#     time_arr.append(time)
    
# print(current_date_dt)
    
# time_arr = []

# # start_time = datetime(hour=0, minute=0)
# # print(start_time)
# # print(type(start_time))
# start_time = datetime.today()
# print(start_time)

# for i in arr:
#     time = start_time
#     time = start_time + i*timedelta(minutes=30)
#     print(time)
#     time_arr.append(time)
    
# # print(time_arr)

# # print(start.strftime('%H_%M'))
# # for i in arr:
    
# # print(arr)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48]
2020-07-05 00:00:00
['00:00', '00:30', '01:00', '01:30', '02:00', '02:30', '03:00', '03:30', '04:00', '04:30', '05:00', '05:30', '06:00', '06:30', '07:00', '07:30', '08:00', '08:30', '09:00', '09:30', '10:00', '10:30', '11:00', '11:30', '12:00', '12:30', '13:00', '13:30', '14:00', '14:30', '15:00', '15:30', '16:00', '16:30', '17:00', '17:30', '18:00', '18:30', '19:00', '19:30', '20:00', '20:30', '21:00', '21:30', '22:00', '22:30', '23:00', '23:30']


In [70]:
df = pd.DataFrame(np.arange(12).reshape(3, 4),
                  columns=['A', 'B', 'C', 'D'])
df = df.drop(columns='A')
df

,B,C,D
0,1,2,3
1,5,6,7
2,9,10,11


In [29]:
a = '年月日,時刻コード,売り入札量(kWh),買い入札量(kWh),約定総量(kWh),システムプライス(円/kWh),エリアプライス北海道(円/kWh),エリアプライス東北(円/kWh),エリアプライス東京(円/kWh),エリアプライス中部(円/kWh),エリアプライス北陸(円/kWh),エリアプライス関西(円/kWh),エリアプライス中国(円/kWh),エリアプライス四国(円/kWh),エリアプライス九州(円/kWh),スポット・時間前平均価格(円/kWh),α上限値×スポット・時間前平均価格(円/kWh),α下限値×スポット・時間前平均価格(円/kWh),α速報値×スポット・時間前平均価格(円/kWh),α確報値×スポット・時間前平均価格(円/kWh),回避可能原価全国値(円/kWh),回避可能原価北海道(円/kWh),回避可能原価東北(円/kWh),回避可能原価東京(円/kWh),回避可能原価中部(円/kWh),回避可能原価北陸(円/kWh),回避可能原価関西(円/kWh),回避可能原価中国(円/kWh),回避可能原価四国(円/kWh),回避可能原価九州(円/kWh)'

c = '年月日	時刻コード	売り入札量(kWh)	買い入札量(kWh)	約定総量(kWh)	システムプライス(円/kWh)	エリアプライス北海道(円/kWh)	エリアプライス東北(円/kWh)	エリアプライス東京(円/kWh)	エリアプライス中部(円/kWh)	エリアプライス北陸(円/kWh)	エリアプライス関西(円/kWh)	エリアプライス中国(円/kWh)	エリアプライス四国(円/kWh)	エリアプライス九州(円/kWh)		スポット・時間前平均価格(円/kWh)	α上限値×スポット・時間前平均価格(円/kWh)	α下限値×スポット・時間前平均価格(円/kWh)	α速報値×スポット・時間前平均価格(円/kWh)	α確報値×スポット・時間前平均価格(円/kWh)		回避可能原価全国値(円/kWh)	回避可能原価北海道(円/kWh)	回避可能原価東北(円/kWh)	回避可能原価東京(円/kWh)	回避可能原価中部(円/kWh)	回避可能原価北陸(円/kWh)	回避可能原価関西(円/kWh)	回避可能原価中国(円/kWh)	回避可能原価四国(円/kWh)	回避可能原価九州(円/kWh)'

def translate_text(str):

    dic = {'年月日': 'date', 
           '時刻コード': 'time code', 
           '売り入札量(kWh)': 'sell bid amount (kWh)', 
           '買い入札量(kWh)': 'buy bid amount (kWh)', 
           '約定総量(kWh)': 'contracted total amount (kWh)', 
           'システムプライス(円/kWh)': 'system price (yen/kWh)', 
           'エリアプライス北海道(円/kWh)': 'area price Hokkaido (yen/kWh)', 
           'エリアプライス東北(円/kWh)': 'area price Tohoku (yen/ kWh)', 
           'エリアプライス東京(円/kWh)': 'area price Tokyo (yen/kWh)', 
           'エリアプライス中部(円/kWh)': 'area price Chubu (yen/kWh)', 
           'エリアプライス北陸(円/kWh)': 'area price Hokuriku (yen/kWh)', 
           'エリアプライス関西(円/kWh)': 'area price Kansai (yen/kWh)', 
           'エリアプライス中国(円/kWh)': 'area price China (yen/kWh)', 
           'エリアプライス四国(円/kWh)': 'area price Shikoku (yen/kWh)', 
           'エリアプライス九州(円/kWh)': 'area price Kyushu (yen/kWh)', 
           'α上限値×スポット・時間前平均価格(円/kWh)': 'upper limit x spot-average price before hour (yen/kWh)', 
           'α下限値×スポット・時間前平均価格(円/kWh)': 'lower limit x spot-average price before hour (yen/kWh)', 
           'α速報値×スポット・時間前平均価格(円/kWh)': 'preliminary value x spot- Hourly average price (yen/kWh)', 
           'α確報値×スポット・時間前平均価格(円/kWh)': 'confirmed value x spot-hourly average price (yen/kWh)', 
           '回避可能原価全国値(円/kWh)': 'national avoidable cost ( JPY/kWh)', 
           '回避可能原価北海道(円/kWh)': 'avoidable cost Hokkaido (JPY/kWh)', 
           '回避可能原価東北(円/kWh)': 'avoidable cost Tohoku (JPY/kWh)', 
           '回避可能原価東京(円/kWh)': 'avoidable cost Tokyo (JPY/kWh)', 
           '回避可能原価中部(円/kWh)': 'avoidable cost Chubu (JPY/kWh)', 
           '回避可能原価北陸(円/kWh)': 'avoidable cost Hokuriku ( JPY/kWh)', 
           '回避可能原価関西(円/kWh)': 'avoidable costs Kansai (JPY/kWh)', 
           '回避可能原価中国(円/kWh)': 'avoidable costs China (JPY/kWh)', 
           '回避可能原価四国(円/kWh)': 'avoidable costs Shikoku (JPY/kWh)', 
           '回避可能原価九州(円/kWh)': 'avoidable costs Kyushu (JPY/kWh)',
           'スポット・時間前平均価格(円/kWh)': 'Spot-average price before hour (yen/kWh)'
          }
    
    for d in dic:
        str = str.replace(d,dic[d])

    return str

format_name(c)

'date\ttime code\tsell bid amount (kWh)\tbuy bid amount (kWh)\tcontracted total amount (kWh)\tsystem price (yen/kWh)\tarea price Hokkaido (yen/kWh)\tarea price Tohoku (yen/ kWh)\tarea price Tokyo (yen/kWh)\tarea price Chubu (yen/kWh)\tarea price Hokuriku (yen/kWh)\tarea price Kansai (yen/kWh)\tarea price China (yen/kWh)\tarea price Shikoku (yen/kWh)\tarea price Kyushu (yen/kWh)\t\tSpot-average price before hour (yen/kWh)\tupper limit x spot-average price before hour (yen/kWh)\tlower limit x spot-average price before hour (yen/kWh)\tpreliminary value x spot- Hourly average price (yen/kWh)\tconfirmed value x spot-hourly average price (yen/kWh)\t\tnational avoidable cost ( JPY/kWh)\tavoidable cost Hokkaido (JPY/kWh)\tavoidable cost Tohoku (JPY/kWh)\tavoidable cost Tokyo (JPY/kWh)\tavoidable cost Chubu (JPY/kWh)\tavoidable cost Hokuriku ( JPY/kWh)\tavoidable costs Kansai (JPY/kWh)\tavoidable costs China (JPY/kWh)\tavoidable costs Shikoku (JPY/kWh)\tavoidable costs Kyushu (JPY/kWh)'

In [ ]:
年月日	時刻コード	売り入札量(kWh)	買い入札量(kWh)	約定総量(kWh)	システムプライス(円/kWh)	エリアプライス北海道(円/kWh)	エリアプライス東北(円/kWh)	エリアプライス東京(円/kWh)	エリアプライス中部(円/kWh)	エリアプライス北陸(円/kWh)	エリアプライス関西(円/kWh)	エリアプライス中国(円/kWh)	エリアプライス四国(円/kWh)	エリアプライス九州(円/kWh)		スポット・時間前平均価格(円/kWh)	α上限値×スポット・時間前平均価格(円/kWh)	α下限値×スポット・時間前平均価格(円/kWh)	α速報値×スポット・時間前平均価格(円/kWh)	α確報値×スポット・時間前平均価格(円/kWh)		回避可能原価全国値(円/kWh)	回避可能原価北海道(円/kWh)	回避可能原価東北(円/kWh)	回避可能原価東京(円/kWh)	回避可能原価中部(円/kWh)	回避可能原価北陸(円/kWh)	回避可能原価関西(円/kWh)	回避可能原価中国(円/kWh)	回避可能原価四国(円/kWh)	回避可能原価九州(円/kWh)
Date,time code,sell bid amount (kWh),buy bid amount (kWh),contracted total amount (kWh),system price (yen/kWh),area price Hokkaido (yen/kWh),area price Tohoku (yen/ kWh),area price Tokyo (yen/kWh),area price Chubu (yen/kWh),area price Hokuriku (yen/kWh),area price Kansai (yen/kWh),area price China (yen/kWh),area price Shikoku (yen/kWh),area price Kyushu (yen/kWh),Spot-average price before hour (yen/kWh),upper limit x spot-average price before hour (yen/kWh),lower limit x spot-average price before hour (yen/kWh),preliminary value x spot- Hourly average price (yen/kWh),confirmed value x spot-hourly average price (yen/kWh),national avoidable cost ( JPY/kWh),avoidable cost Hokkaido (JPY/kWh),avoidable cost Tohoku (JPY/kWh),avoidable cost Tokyo (JPY/kWh),avoidable cost Chubu (JPY/kWh),avoidable cost Hokuriku ( JPY/kWh),avoidable costs Kansai (JPY/kWh),avoidable costs China (JPY/kWh),avoidable costs Shikoku (JPY/kWh),avoidable costs Kyushu (JPY/kWh)


In [26]:
a = '年月日	時刻コード	売り入札量(kWh)	買い入札量(kWh)	約定総量(kWh)	システムプライス(円/kWh)	エリアプライス北海道(円/kWh)	エリアプライス東北(円/kWh)	エリアプライス東京(円/kWh)	エリアプライス中部(円/kWh)	エリアプライス北陸(円/kWh)	エリアプライス関西(円/kWh)	エリアプライス中国(円/kWh)	エリアプライス四国(円/kWh)	エリアプライス九州(円/kWh)		スポット・時間前平均価格(円/kWh)	α上限値×スポット・時間前平均価格(円/kWh)	α下限値×スポット・時間前平均価格(円/kWh)	α速報値×スポット・時間前平均価格(円/kWh)	α確報値×スポット・時間前平均価格(円/kWh)		回避可能原価全国値(円/kWh)	回避可能原価北海道(円/kWh)	回避可能原価東北(円/kWh)	回避可能原価東京(円/kWh)	回避可能原価中部(円/kWh)	回避可能原価北陸(円/kWh)	回避可能原価関西(円/kWh)	回避可能原価中国(円/kWh)	回避可能原価四国(円/kWh)	回避可能原価九州(円/kWh)'

b = '年月日	時刻コード	売り入札量(kWh)	買い入札量(kWh)	約定総量(kWh)	システムプライス(円/kWh)	エリアプライス北海道(円/kWh)	エリアプライス東北(円/kWh)	エリアプライス東京(円/kWh)	エリアプライス中部(円/kWh)	エリアプライス北陸(円/kWh)	エリアプライス関西(円/kWh)	エリアプライス中国(円/kWh)	エリアプライス四国(円/kWh)	エリアプライス九州(円/kWh)		スポット・時間前平均価格(円/kWh)	α上限値×スポット・時間前平均価格(円/kWh)	α下限値×スポット・時間前平均価格(円/kWh)	α速報値×スポット・時間前平均価格(円/kWh)	α確報値×スポット・時間前平均価格(円/kWh)		回避可能原価全国値(円/kWh)	回避可能原価北海道(円/kWh)	回避可能原価東北(円/kWh)	回避可能原価東京(円/kWh)	回避可能原価中部(円/kWh)	回避可能原価北陸(円/kWh)	回避可能原価関西(円/kWh)	回避可能原価中国(円/kWh)	回避可能原価四国(円/kWh)	回避可能原価九州(円/kWh)'

print(a==b)

True


In [44]:
df = pd.DataFrame()
# df
df2 = [5, 6]
# df2 = pd.DataFrame([[5, 6], [7, 8]])
# print(df2)
df = df.append([df2])

df

# df3 = pd.DataFrame([[5, 5], [4, 4]], columns=list('AB'))

# df = df.append(df3, ignore_index=True)
df

,0,1
0,5,6


In [16]:
import numpy as np
df = pd.DataFrame({"name": ['Alfred', 'Batman', 'Catwoman'],
                   "toy": [np.nan, 'Batmobile', 'Bullwhip'],
                   "born": [np.nan, np.nan,
                            np.nan]})

df = df.dropna(how='all', axis = 1)
df

,name,toy
0,Alfred,NaN
1,Batman,Batmobile
2,Catwoman,Bullwhip
